<p style='margin: 12px; font-size: 36px; text-align:center'>Driving LLDB from Python</p>

Locating the python lldb module:

* On mac, when brewed, `lldb -P` works
* On Ubuntu-20.4, it's wrong with lldb-12 at least. You can try to locate the "embedded_interpreter.py" instead

In [1]:
!lldb -P

/usr/lib/lib/python3/dist-packages


In [5]:
# or `find /usr/ -name "embedded_interpreter.py"`
!mlocate "embedded_interpreter.py"

/usr/lib/llvm-12/lib/python3/dist-packages/lldb/embedded_interpreter.py


In [15]:
import os
from pathlib import Path
import platform
import sys

if platform.system() == 'Linux':
    lldb_path = '/usr/lib/llvm-12/lib/python3/dist-packages'
elif platform.system() == 'Darwin':
    lldb_path = '/opt/homebrew/Cellar/llvm/14.0.6_1/libexec/python3.10/site-packages'
else:
    raise ValueError("Dunno where to find lldb, locate it yourself")

sys.path.insert(0, lldb_path)
import lldb

https://github.com/llvm/llvm-project/blob/main/lldb/packages/Python/lldbsuite/test/lldbutil.py

In [2]:
from enum import Enum

class LLDBState(Enum):
    Invalid = lldb.eStateInvalid
    Unloaded = lldb.eStateUnloaded
    Connected = lldb.eStateConnected
    Attaching = lldb.eStateAttaching
    Launching = lldb.eStateLaunching
    Stopped = lldb.eStateStopped
    Running = lldb.eStateRunning
    Stepping = lldb.eStateStepping
    Crashed = lldb.eStateCrashed
    Detached = lldb.eStateDetached
    Exited = lldb.eStateExited
    Suspended = lldb.eStateSuspended

In [3]:
def display_frame(thread: lldb.SBThread):
    frame = thread.GetSelectedFrame()
    debugger = thread.GetProcess().GetTarget().GetDebugger()
    source_mgr = debugger.GetSourceManager()
    # Use a string stream as the destination.
    stream = lldb.SBStream()
    lineEntry = frame.GetLineEntry()
    line_num = lineEntry.GetLine()
    fileSpec = lineEntry.GetFileSpec()
    source_mgr.DisplaySourceLinesWithLineNumbers(fileSpec,
                                                 line_num,
                                                 10, # context before
                                                 10, # context after
                                                 '=>', # prefix for current line
                                                 stream)
    print(stream.GetData())

In [4]:
def display_breakpoints(target: lldb.SBTarget):
    print(f"There are {target.num_breakpoints} breakpoints\n")
    for b in target.breakpoint_iter():
        print(b)
        for l in b.get_breakpoint_location_list():
            print("  " + str(l))
        print("")

# Basic example

Compile a dumb cpp file into an excutable in the build directory

In [5]:
out_dir = Path('build').resolve()
out_dir.mkdir(exist_ok=True)

In [6]:
!g++ -std=c++20 -O0 -g main.cpp -o build/test && ./build/test

1 0 


In [7]:
exe = str(Path('build/test').resolve())

In [8]:
# Create a new debugger instance
debugger = lldb.SBDebugger.Create()

# We want pretty colors!
debugger.SetUseColor(True)

# When we step or continue, don't return from the function until the process
# stops. We do this by setting the async mode to false.
debugger.SetAsync(False)

# Create a target from a file and arch
print(f"Creating a target for '{exe}'")

target = debugger.CreateTargetWithFileAndArch (exe, lldb.LLDB_ARCH_DEFAULT)

if not target:
    raise ValueError(f"Failed to create a target for '{exe}'")
    
# If the target is valid set a breakpoint at main
fname = 'main'
main_bp = target.BreakpointCreateByName (fname, target.GetExecutable().GetFilename())
print(main_bp)

Creating a target for '/Users/julien/Software/LldbScripting/build/test'
SBBreakpoint: id = 1, name = 'main', module = test, locations = 1


In [9]:
process = target.LaunchSimple(argv=None, envp=None, working_directory=os.getcwd())
assert process

# Print some simple process info
state = process.GetState()
LLDBState(state)

<LLDBState.Stopped: 5>

In [10]:
thread = process.GetSelectedThread()
assert thread

print(f"There are {thread.GetNumFrames()} Frames")
for frame in thread.get_thread_frames():
    print(frame)

There are 2 Frames
frame #0: 0x0000000100000efc test`main at main.cpp:6:31
frame #1: 0x0000000100029088 dyld`start + 516


In [11]:
# Get the current frame
frame = thread.GetSelectedFrame()
assert(frame)
print(frame)

frame #0: 0x0000000100000efc test`main at main.cpp:6:31


In [12]:
source_mgr = debugger.GetSourceManager()
# Use a string stream as the destination.
stream = lldb.SBStream()
lineEntry = frame.GetLineEntry()
line_num = lineEntry.GetLine()
fileSpec = lineEntry.GetFileSpec()
source_mgr.DisplaySourceLinesWithLineNumbers(fileSpec,
                                             line_num,
                                             10, # context before
                                             10, # context after
                                             '=>', # prefix for current line
                                             stream)
print(stream.GetData())

   1   	#include <iostream>
   2   	#include <string>
   3   	#include <vector>
   4   	
   5   	int main() {
=> 6   	  std::vector<std::string> vec{"First string", "Second String that is longer"};
   7   	  std::vector<int> intVec;
   8   	  intVec.resize(vec.size(), -1);
   9   	  std::vector<bool> moved;
   10  	  moved.resize(vec.size(), false);
   11  	
   12  	  int newPos = 0;
   13  	  for (int i = 0; auto& s : vec) {
   14  	    if (s.size() > 20) {
   15  	      moved[i] = true;
   16  	      intVec[newPos++] = i;
   17  	    }
   18  	    ++i;
   19  	  }
   20  	
   21  	  for (int i = 0; auto& s : vec) {



In [13]:
thread.StepOver()
assert LLDBState(process.GetState()) == LLDBState.Stopped
display_frame(thread)

   1   	#include <iostream>
   2   	#include <string>
   3   	#include <vector>
   4   	
   5   	int main() {
   6   	  std::vector<std::string> vec{"First string", "Second String that is longer"};
=> 7   	  std::vector<int> intVec;
   8   	  intVec.resize(vec.size(), -1);
   9   	  std::vector<bool> moved;
   10  	  moved.resize(vec.size(), false);
   11  	
   12  	  int newPos = 0;
   13  	  for (int i = 0; auto& s : vec) {
   14  	    if (s.size() > 20) {
   15  	      moved[i] = true;
   16  	      intVec[newPos++] = i;
   17  	    }
   18  	    ++i;
   19  	  }
   20  	
   21  	  for (int i = 0; auto& s : vec) {



In [14]:
thread.StepOver()
assert LLDBState(process.GetState()) == LLDBState.Stopped
display_frame(thread)

   1   	#include <iostream>
   2   	#include <string>
   3   	#include <vector>
   4   	
   5   	int main() {
   6   	  std::vector<std::string> vec{"First string", "Second String that is longer"};
   7   	  std::vector<int> intVec;
=> 8   	  intVec.resize(vec.size(), -1);
   9   	  std::vector<bool> moved;
   10  	  moved.resize(vec.size(), false);
   11  	
   12  	  int newPos = 0;
   13  	  for (int i = 0; auto& s : vec) {
   14  	    if (s.size() > 20) {
   15  	      moved[i] = true;
   16  	      intVec[newPos++] = i;
   17  	    }
   18  	    ++i;
   19  	  }
   20  	
   21  	  for (int i = 0; auto& s : vec) {



In [15]:
process.Continue()
assert LLDBState(process.GetState()) == LLDBState.Exited

# More complex example

## Create a debugger and a target

In [16]:
exe = '/Users/julien/Software/Others/EnergyPlus-build/Products/energyplus'

In [17]:
# Create a new debugger instance
debugger = lldb.SBDebugger.Create()

# We want pretty colors!
debugger.SetUseColor(True)

# When we step or continue, don't return from the function until the process
# stops. We do this by setting the async mode to false.
debugger.SetAsync(False)

# Create a target from a file and arch
print(f"Creating a target for '{exe}'")

target = debugger.CreateTargetWithFileAndArch (exe, lldb.LLDB_ARCH_DEFAULT)

if not target:
    raise ValueError(f"Failed to create a target for '{exe}'")

Creating a target for '/Users/julien/Software/Others/EnergyPlus-build/Products/energyplus'


In [18]:
target.GetExecutable().GetFilename()

'energyplus'

## Breakpoints

In [19]:
fname = 'CreateMissingSpaces'
bp1 = target.BreakpointCreateByName (fname) # , target.GetExecutable().GetFilename())
print(bp1)

SBBreakpoint: id = 1, name = 'CreateMissingSpaces', locations = 1


In [20]:
bp2 = target.BreakpointCreateByLocation('SurfaceGeometry.cc', 1566)
print(bp2)

SBBreakpoint: id = 2, file = 'SurfaceGeometry.cc', line = 1566, exact_match = 0, locations = 1


In [21]:
display_breakpoints(target=target)

There are 2 breakpoints

SBBreakpoint: id = 1, name = 'CreateMissingSpaces', locations = 1
  1.1: where = libenergyplusapi.22.2.0.dylib`EnergyPlus::SurfaceGeometry::CreateMissingSpaces(EnergyPlus::EnergyPlusData&, bool&) + 36 at SurfaceGeometry.cc:2884:18, address = libenergyplusapi.22.2.0.dylib[0x0000000002372908], unresolved, hit count = 0 

SBBreakpoint: id = 2, file = 'SurfaceGeometry.cc', line = 1566, exact_match = 0, locations = 1
  2.1: where = libenergyplusapi.22.2.0.dylib`EnergyPlus::SurfaceGeometry::GetSurfaceData(EnergyPlus::EnergyPlusData&, bool&) + 15872 at SurfaceGeometry.cc:1566:33, address = libenergyplusapi.22.2.0.dylib[0x0000000002320248], unresolved, hit count = 0 



## Start process with args

In [22]:
debugger.HandleCommand("settings set target.process.thread.step-out-avoid-nodebug true")

In [23]:
epw = '/Users/julien/Software/Others/EnergyPlus/weather/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw'
idf = '/Users/julien/Software/Others/EnergyPlus/testfiles/DaylightingDeviceTubular.idf'

In [24]:
argv = ["-D", "-r", "-w", epw, "-d", "out-temp", idf] 
print(" ".join(argv))

-D -r -w /Users/julien/Software/Others/EnergyPlus/weather/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw -d out-temp /Users/julien/Software/Others/EnergyPlus/testfiles/DaylightingDeviceTubular.idf


In [25]:
# Launch the process. Since we specified synchronous mode, we won't return
# from this function until we hit the breakpoint at main
process = target.LaunchSimple(argv=argv, envp=None, working_directory=os.getcwd())

# Make sure the launch went ok
assert process

In [26]:
print(process.GetSTDOUT(1000000))

EnergyPlus Starting
EnergyPlus, Version 22.2.0-d0267d6daa, YMD=2022.08.03 10:42
Adjusting Air System Sizing
Adjusting Standard 62.1 Ventilation Sizing
Initializing Simulation



In [27]:
# Make sure the launch went ok
if process:
    # Print some simple process info
    state = process.GetState()
    print(process)

SBProcess: pid = 40770, state = stopped, threads = 1, executable = energyplus


In [31]:
assert state == lldb.eStateStopped
assert LLDBState(state) == LLDBState.Stopped

In [32]:
bp1.GetNumLocations(), bp1.GetNumResolvedLocations(), bp1.IsEnabled()

(1, 1, True)

In [33]:
bp2.GetNumLocations(), bp2.GetNumResolvedLocations(), bp2.IsEnabled()

(1, 1, True)

In [34]:
for bl in bp2:
    print('breakpoint location load addr: %s' % hex(bl.GetLoadAddress()))
    if bl.GetCondition():
        print('breakpoint location condition: %s' % hex(bl.GetCondition()))

breakpoint location load addr: 0x10c7ec248


In [35]:
# Get the first thread
# thread = process.GetThreadAtIndex(0)
# Get the current thread
thread = process.GetSelectedThread()
assert thread

In [36]:
print(thread)

thread #1: tid = 0xe3bd6, 0x000000010c7ec248 libenergyplusapi.22.2.0.dylib`EnergyPlus::SurfaceGeometry::GetSurfaceData(state=0x000000016fdfd180, ErrorsFound=0x000000016fdfc687) at SurfaceGeometry.cc:1566:33, queue = 'com.apple.main-thread', stop reason = breakpoint 2.1


In [37]:
thread.GetNumFrames()

11

In [38]:
for frame in thread.get_thread_frames():
    print(frame)

frame #0: 0x000000010c7ec248 libenergyplusapi.22.2.0.dylib`EnergyPlus::SurfaceGeometry::GetSurfaceData(state=0x000000016fdfd180, ErrorsFound=0x000000016fdfc687) at SurfaceGeometry.cc:1566:33
frame #1: 0x000000010c7e4968 libenergyplusapi.22.2.0.dylib`EnergyPlus::SurfaceGeometry::SetupZoneGeometry(state=0x000000016fdfd180, ErrorsFound=0x000000016fdfc687) at SurfaceGeometry.cc:351:9
frame #2: 0x000000010b70dc2c libenergyplusapi.22.2.0.dylib`EnergyPlus::HeatBalanceManager::GetBuildingData(state=0x000000016fdfd180, ErrorsFound=0x000000016fdfc687) at HeatBalanceManager.cc:4910:9
frame #3: 0x000000010b6c45f0 libenergyplusapi.22.2.0.dylib`EnergyPlus::HeatBalanceManager::GetHeatBalanceInput(state=0x000000016fdfd180) at HeatBalanceManager.cc:328:9
frame #4: 0x000000010b6c4128 libenergyplusapi.22.2.0.dylib`EnergyPlus::HeatBalanceManager::ManageHeatBalance(state=0x000000016fdfd180) at HeatBalanceManager.cc:204:13
frame #5: 0x000000010c5aa560 libenergyplusapi.22.2.0.dylib`EnergyPlus::SimulationMana

In [39]:
# Get the first frame
# frame = thread.GetFrameAtIndex(0)
# Get the current frame
frame = thread.GetSelectedFrame()
assert(frame)
print(frame)

frame #0: 0x000000010c7ec248 libenergyplusapi.22.2.0.dylib`EnergyPlus::SurfaceGeometry::GetSurfaceData(state=0x000000016fdfd180, ErrorsFound=0x000000016fdfc687) at SurfaceGeometry.cc:1566:33


In [40]:
source_mgr = debugger.GetSourceManager()
# Use a string stream as the destination.
stream = lldb.SBStream()
lineEntry = frame.GetLineEntry()
line_num = lineEntry.GetLine()
fileSpec = lineEntry.GetFileSpec()
source_mgr.DisplaySourceLinesWithLineNumbers(fileSpec,
                                             line_num,
                                             10, # context before
                                             10, # context after
                                             '=>', # prefix for current line
                                             stream)
print(stream.GetData())

   1556	        //    After reordering, MovedSurfs should equal TotSurfaces
   1557	
   1558	        MovedSurfs = 0;
   1559	        Array1D<bool> SurfaceTmpClassMoved; // Tmp class is moved
   1560	        SurfaceTmpClassMoved.dimension(state.dataSurface->TotSurfaces, false);
   1561	
   1562	        // Old SurfNum to New SurfNum
   1563	        // Old = order in state.dataSurfaceGeometry->SurfaceTmp
   1564	        // New = order in state.dataSurface->Surface
   1565	        EPVector<int> oldToNewSurfNums;
=> 1566	        oldToNewSurfNums.resize(state.dataSurface->TotSurfaces, -1);
   1567	
   1568	        // Move all shading Surfaces to Front
   1569	        for (int SurfNum = 1; SurfNum <= state.dataSurface->TotSurfaces; ++SurfNum) {
   1570	            if (state.dataSurfaceGeometry->SurfaceTmp(SurfNum).Class != SurfaceClass::Detached_F &&
   1571	                state.dataSurfaceGeometry->SurfaceTmp(SurfNum).Class != SurfaceClass::Detached_B &&
   1572	                state.dataSu

## Retrieve array data into a nice python dict

In [41]:
root = frame.FindVariable("state")
assert(root.error.success)
surfaces = root.GetChildMemberWithName('dataSurfaceGeometry').Dereference().GetChildMemberWithName('SurfaceTmp')
assert(surfaces.error.success)
n_surfaces = surfaces.GetChildMemberWithName('size_').GetValueAsUnsigned()


surfaces_ptr = surfaces.GetChildMemberWithName('data_')
surfaceDataType = surfaces_ptr.GetType().GetPointeeType()
surfaceByteSize = surfaceDataType.GetByteSize()   # 1068


construct = root.GetChildMemberWithName('dataConstruction').Dereference().GetChildMemberWithName('Construct')
assert(construct.error.success)
construct_ptr = construct.GetChildMemberWithName('data_')
constructDataType = construct_ptr.GetType().GetPointeeType()
constructByteSize = constructDataType.GetByteSize()   # 7064


def get_and_format_as_string(val, childName):
    return str(val.GetChildMemberWithName(childName)).split(f"{childName} = ")[1].replace('"', '').strip()

infos = []
for i in range (0, n_surfaces):
    offset = surfaces_ptr.GetValueAsUnsigned() + i * surfaceByteSize
    val = target.CreateValueFromAddress("temp", lldb.SBAddress(offset, target), surfaceDataType)
    name =  get_and_format_as_string(val, 'Name')
    zone = val.GetChildMemberWithName('Zone').GetValueAsSigned()
    className = get_and_format_as_string(val, 'Class')
    classInt = val.GetChildMemberWithName('Class').GetValueAsSigned()
    baseSurf = val.GetChildMemberWithName('BaseSurf').GetValueAsSigned()
    baseSurfName = get_and_format_as_string(val, 'BaseSurfName')
    extBoundCond = val.GetChildMemberWithName('ExtBoundCond').GetValueAsSigned()
    extBoundCondName = get_and_format_as_string(val, 'ExtBoundCondName')
    heatTransSurf = val.GetChildMemberWithName('HeatTransSurf').GetValueAsSigned() == 1
    
    construction = val.GetChildMemberWithName('Construction').GetValueAsSigned()
    c_offset = construct_ptr.GetValueAsUnsigned() + (construction - 1) * constructByteSize
    c_val = target.CreateValueFromAddress("temp", lldb.SBAddress(c_offset, target), constructDataType)
    c_name = get_and_format_as_string(c_val, 'Name')
    typeBound = c_val.GetChildMemberWithName('TypeIsAirBoundary').GetValueAsSigned() == 1
    
    print(f"1-index={i+1}, {name=}, {zone=}, {className=}, {baseSurf=}, {baseSurfName=}, {extBoundCond=}, {extBoundCondName=}")
    infos.append({'SurfNum': i + 1, 'Name': name, 'Zone': zone, 'ClassInt': classInt, 'BaseSurf': baseSurf, 'BaseSurfName': baseSurfName,
                  'ExtBoundCond': extBoundCond, 'ExtBoundCondName': extBoundCondName, 'HeatTransSurf': heatTransSurf,
                  'Construction': construction, 'ConstructionName': c_name, 'TypeIsAirBoundary': typeBound})

1-index=1, name='DAYLIT SOUTH WALL', zone=1, className='Wall', baseSurf=1, baseSurfName='DAYLIT SOUTH WALL', extBoundCond=0, extBoundCondName=''
1-index=2, name='DAYLIT WEST WALL', zone=1, className='Wall', baseSurf=2, baseSurfName='DAYLIT WEST WALL', extBoundCond=0, extBoundCondName=''
1-index=3, name='DAYLIT NORTH WALL', zone=1, className='Wall', baseSurf=3, baseSurfName='DAYLIT NORTH WALL', extBoundCond=0, extBoundCondName=''
1-index=4, name='DAYLIT EAST WALL', zone=1, className='Wall', baseSurf=4, baseSurfName='DAYLIT EAST WALL', extBoundCond=0, extBoundCondName=''
1-index=5, name='DAYLIT FLOOR', zone=1, className='Floor', baseSurf=5, baseSurfName='DAYLIT FLOOR', extBoundCond=-999, extBoundCondName='DAYLIT FLOOR'
1-index=6, name='DAYLIT CEILING', zone=1, className='Roof', baseSurf=6, baseSurfName='DAYLIT CEILING', extBoundCond=-999, extBoundCondName='DAYLIT ATTIC FLOOR'
1-index=7, name='DAYLIT ATTIC SOUTH WALL', zone=2, className='Wall', baseSurf=7, baseSurfName='DAYLIT ATTIC SOUTH

In [43]:
infos[:2]

[{'SurfNum': 1,
  'Name': 'DAYLIT SOUTH WALL',
  'Zone': 1,
  'ClassInt': 1,
  'BaseSurf': 1,
  'BaseSurfName': 'DAYLIT SOUTH WALL',
  'ExtBoundCond': 0,
  'ExtBoundCondName': '',
  'HeatTransSurf': True,
  'Construction': 1,
  'ConstructionName': 'EXTWALL80',
  'TypeIsAirBoundary': False},
 {'SurfNum': 2,
  'Name': 'DAYLIT WEST WALL',
  'Zone': 1,
  'ClassInt': 1,
  'BaseSurf': 2,
  'BaseSurfName': 'DAYLIT WEST WALL',
  'ExtBoundCond': 0,
  'ExtBoundCondName': '',
  'HeatTransSurf': True,
  'Construction': 1,
  'ConstructionName': 'EXTWALL80',
  'TypeIsAirBoundary': False}]

## Step over

In [44]:
thread.StepOver()

display_frame(thread)

   1559	        Array1D<bool> SurfaceTmpClassMoved; // Tmp class is moved
   1560	        SurfaceTmpClassMoved.dimension(state.dataSurface->TotSurfaces, false);
   1561	
   1562	        // Old SurfNum to New SurfNum
   1563	        // Old = order in state.dataSurfaceGeometry->SurfaceTmp
   1564	        // New = order in state.dataSurface->Surface
   1565	        EPVector<int> oldToNewSurfNums;
   1566	        oldToNewSurfNums.resize(state.dataSurface->TotSurfaces, -1);
   1567	
   1568	        // Move all shading Surfaces to Front
=> 1569	        for (int SurfNum = 1; SurfNum <= state.dataSurface->TotSurfaces; ++SurfNum) {
   1570	            if (state.dataSurfaceGeometry->SurfaceTmp(SurfNum).Class != SurfaceClass::Detached_F &&
   1571	                state.dataSurfaceGeometry->SurfaceTmp(SurfNum).Class != SurfaceClass::Detached_B &&
   1572	                state.dataSurfaceGeometry->SurfaceTmp(SurfNum).Class != SurfaceClass::Shading)
   1573	                continue;
   1574	
   1575

In [45]:
thread.JumpToLine(fileSpec, 1615)
display_frame(thread)

   1605	                    state.dataSurface->AllSurfaceListReportOrder.push_back(SurfNum);
   1606	                }
   1607	                oldToNewSurfNums(SurfNum) = MovedSurfs;
   1608	                SurfaceTmpClassMoved(SurfNum) = true; //'Moved'
   1609	            }
   1610	
   1611	            //  For each Base Surface Type (Wall, Floor, Roof/Ceiling) - put these first
   1612	
   1613	            for (int Loop = 1; Loop <= 3; ++Loop) {
   1614	
=> 1615	                for (int SurfNum = 1; SurfNum <= state.dataSurface->TotSurfaces; ++SurfNum) {
   1616	
   1617	                    if (SurfaceTmpClassMoved(SurfNum)) continue;
   1618	                    if (state.dataSurfaceGeometry->SurfaceTmp(SurfNum).Zone == 0) continue;
   1619	
   1620	                    if (!UtilityRoutines::SameString(state.dataSurfaceGeometry->SurfaceTmp(SurfNum).ZoneName, state.dataHeatBal->Zone(ZoneNum).Name))
   1621	                        continue;
   1622	                    if (state.dataSurf

In [49]:
thread.JumpToLine(fileSpec, 1772)
display_frame(thread)

   1762	                                    std::string{RoutineName} + "Error in Surface= \"" + state.dataSurfaceGeometry->SurfaceTmp(Loop).Name +
   1763	                                        " indicated Zone=\"" + state.dataSurfaceGeometry->SurfaceTmp(Loop).ZoneName + "\"");
   1764	                }
   1765	            }
   1766	            ShowWarningError(state,
   1767	                             std::string{RoutineName} +
   1768	                                 "Remaining surface checks will use \"reordered number of surfaces\", not number of original surfaces");
   1769	        }
   1770	
   1771	        // Realign the relationship: surface to base surface
=> 1772	        for (int SurfNum = 1; SurfNum <= state.dataSurface->TotSurfaces; ++SurfNum) {
   1773	            auto &movedSurf = state.dataSurface->Surface(SurfNum);
   1774	            if (movedSurf.BaseSurf > 0) {
   1775	                int newBaseSurfNum = oldToNewSurfNums(movedSurf.BaseSurf);
   1776	             

In [50]:
oldToNewSurfNums = frame.FindVariable("oldToNewSurfNums")
assert(oldToNewSurfNums.error.success)
assert(oldToNewSurfNums.GetTypeName() == 'EnergyPlus::EPVector<int>')
vec = oldToNewSurfNums.GetChildAtIndex(0)

In [52]:
print(vec)

(std::vector<int, std::allocator<int> >) std::__1::vector<int, std::__1::allocator<int> > = size=28 {
  [0] = -1
  [1] = -1
  [2] = -1
  [3] = -1
  [4] = -1
  [5] = -1
  [6] = -1
  [7] = -1
  [8] = -1
  [9] = -1
  [10] = -1
  [11] = -1
  [12] = -1
  [13] = -1
  [14] = -1
  [15] = -1
  [16] = -1
  [17] = -1
  [18] = -1
  [19] = -1
  [20] = -1
  [21] = -1
  [22] = -1
  [23] = -1
  [24] = -1
  [25] = -1
  [26] = -1
  [27] = -1
}


In [53]:
for i in vec.children:
    print(i.GetValueAsSigned())

-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1
-1


## Disassemble / print registers

In [54]:
function = frame.GetFunction()
# See if we have debug info (a function)
assert function
print(function)

SBFunction: id = 0xed0011dfa6, name = EnergyPlus::SurfaceGeometry::GetSurfaceData(EnergyPlus::EnergyPlusData&, bool&), type = GetSurfaceData


In [55]:
print(function.GetInstructions(target))

libenergyplusapi.22.2.0.dylib[0x231c448]: stp    x28, x27, [sp, #-0x20]!
libenergyplusapi.22.2.0.dylib[0x231c44c]: stp    x29, x30, [sp, #0x10]
libenergyplusapi.22.2.0.dylib[0x231c450]: add    x29, sp, #0x10
libenergyplusapi.22.2.0.dylib[0x231c454]: mov    w9, #0x6290
libenergyplusapi.22.2.0.dylib[0x231c458]: adrp   x16, 8016
libenergyplusapi.22.2.0.dylib[0x231c45c]: ldr    x16, [x16, #0xdb8]
libenergyplusapi.22.2.0.dylib[0x231c460]: blr    x16
libenergyplusapi.22.2.0.dylib[0x231c464]: sub    sp, sp, #0x6, lsl #12     ; =0x6000 
libenergyplusapi.22.2.0.dylib[0x231c468]: sub    sp, sp, #0x290
libenergyplusapi.22.2.0.dylib[0x231c46c]: add    x8, sp, #0x5, lsl #12     ; =0x5000 
libenergyplusapi.22.2.0.dylib[0x231c470]: add    x8, x8, #0xaa7
libenergyplusapi.22.2.0.dylib[0x231c474]: str    x8, [sp, #0x25d0]
libenergyplusapi.22.2.0.dylib[0x231c478]: adrp   x9, 8222
libenergyplusapi.22.2.0.dylib[0x231c47c]: add    x9, x9, #0x120
libenergyplusapi.22.2.0.dylib[0x231c480]: str    x9, [sp, #0x2

In [56]:
def disassemble_instructions (insts):
    for i in insts:
        print (i)

if frame:
    # Print some simple frame info
    print(frame)
    function = frame.GetFunction()
    # See if we have debug info (a function)
    if function:
        # We do have a function, print some info for the function
        print(function)
        # Now get all instructions for this function and print them
        insts = function.GetInstructions(target)
        disassemble_instructions (insts)
    else:
        # See if we have a symbol in the symbol table for where we stopped
        symbol = frame.GetSymbol();
        if symbol:
            # We do have a symbol, print some info for the symbol
            print(symbol)
            # Now get all instructions for this symbol and print them
            insts = symbol.GetInstructions(target)
            disassemble_instructions (insts)

    registerList = frame.GetRegisters()
    print('Frame registers (size of register set = %d):' % registerList.GetSize())
    for value in registerList:
        #print value
        print('%s (number of children = %d):' % (value.GetName(), value.GetNumChildren()))
        for child in value:
            print('Name: ', child.GetName(), ' Value: ', child.GetValue())

frame #0: 0x000000010c7ee528 libenergyplusapi.22.2.0.dylib`EnergyPlus::SurfaceGeometry::GetSurfaceData(state=0x000000016fdfd180, ErrorsFound=0x000000016fdfc687) at SurfaceGeometry.cc:1772:18
SBFunction: id = 0xed0011dfa6, name = EnergyPlus::SurfaceGeometry::GetSurfaceData(EnergyPlus::EnergyPlusData&, bool&), type = GetSurfaceData
libenergyplusapi.22.2.0.dylib[0x231c448]: stp    x28, x27, [sp, #-0x20]!
libenergyplusapi.22.2.0.dylib[0x231c44c]: stp    x29, x30, [sp, #0x10]
libenergyplusapi.22.2.0.dylib[0x231c450]: add    x29, sp, #0x10
libenergyplusapi.22.2.0.dylib[0x231c454]: mov    w9, #0x6290
libenergyplusapi.22.2.0.dylib[0x231c458]: adrp   x16, 8016
libenergyplusapi.22.2.0.dylib[0x231c45c]: ldr    x16, [x16, #0xdb8]
libenergyplusapi.22.2.0.dylib[0x231c460]: blr    x16
libenergyplusapi.22.2.0.dylib[0x231c464]: sub    sp, sp, #0x6, lsl #12     ; =0x6000 
libenergyplusapi.22.2.0.dylib[0x231c468]: sub    sp, sp, #0x290
libenergyplusapi.22.2.0.dylib[0x231c46c]: add    x8, sp, #0x5, lsl #1

libenergyplusapi.22.2.0.dylib[0x2327ce0]: ldr    w8, [x8, #0x3c]
libenergyplusapi.22.2.0.dylib[0x2327ce4]: subs   w8, w8, #0x2
libenergyplusapi.22.2.0.dylib[0x2327ce8]: b.ne   0x2327cf4
libenergyplusapi.22.2.0.dylib[0x2327cec]: b      0x2327cf0
libenergyplusapi.22.2.0.dylib[0x2327cf0]: b      0x232ab70
libenergyplusapi.22.2.0.dylib[0x2327cf4]: b      0x2327cf8
libenergyplusapi.22.2.0.dylib[0x2327cf8]: ldur   x8, [x29, #-0x18]
libenergyplusapi.22.2.0.dylib[0x2327cfc]: add    x0, x8, #0xea8
libenergyplusapi.22.2.0.dylib[0x2327d00]: bl     0x231835c
libenergyplusapi.22.2.0.dylib[0x2327d04]: mov    x8, #0x5798
libenergyplusapi.22.2.0.dylib[0x2327d08]: add    x0, x0, x8
libenergyplusapi.22.2.0.dylib[0x2327d0c]: add    x8, sp, #0x1, lsl #12     ; =0x1000 
libenergyplusapi.22.2.0.dylib[0x2327d10]: add    x8, x8, #0x758
libenergyplusapi.22.2.0.dylib[0x2327d14]: ldrsw  x1, [x8, #0x3ffc]
libenergyplusapi.22.2.0.dylib[0x2327d18]: bl     0x357d408
libenergyplusapi.22.2.0.dylib[0x2327d1c]: str    x

libenergyplusapi.22.2.0.dylib[0x233384c]: add    x0, x0, #0x748
libenergyplusapi.22.2.0.dylib[0x2333850]: bl     0x10374
libenergyplusapi.22.2.0.dylib[0x2333854]: b      0x2333858
libenergyplusapi.22.2.0.dylib[0x2333858]: add    x0, sp, #0x2, lsl #12     ; =0x2000 
libenergyplusapi.22.2.0.dylib[0x233385c]: add    x0, x0, #0x730
libenergyplusapi.22.2.0.dylib[0x2333860]: bl     0x10374
libenergyplusapi.22.2.0.dylib[0x2333864]: b      0x2333868
libenergyplusapi.22.2.0.dylib[0x2333868]: ldr    x0, [sp, #0x138]
libenergyplusapi.22.2.0.dylib[0x233386c]: add    x1, sp, #0x2, lsl #12     ; =0x2000 
libenergyplusapi.22.2.0.dylib[0x2333870]: add    x1, x1, #0x758
libenergyplusapi.22.2.0.dylib[0x2333874]: add    x2, sp, #0x2, lsl #12     ; =0x2000 
libenergyplusapi.22.2.0.dylib[0x2333878]: add    x2, x2, #0x748
libenergyplusapi.22.2.0.dylib[0x233387c]: add    x3, sp, #0x2, lsl #12     ; =0x2000 
libenergyplusapi.22.2.0.dylib[0x2333880]: add    x3, x3, #0x730
libenergyplusapi.22.2.0.dylib[0x233388

In [57]:
process.Continue()

<lldb.SBError; proxy of <Swig Object of type 'lldb::SBError *' at 0x1578e1f20> >

In [58]:
LLDBState(process.GetState())

<LLDBState.Stopped: 5>

In [61]:
LLDBState(process.GetState())

<LLDBState.Stopped: 5>

In [62]:
process.Continue()

<lldb.SBError; proxy of <Swig Object of type 'lldb::SBError *' at 0x127fdb930> >

In [60]:
if state == lldb.eStateExited:
    print("program has exited...")
else:
    print('Unexpected process state: %s, killing process...' % debugger.StateAsCString (state))
    process.Kill()

Unexpected process state: stopped, killing process...
